In [19]:
#To scrap all the pdfs from CCI website

import requests
from bs4 import BeautifulSoup
import io
import datetime
import csv
from PyPDF2 import PdfFileReader #used the PyPDF2 < 3.0
import pandas as pd
from urllib.parse import urljoin
import re
import os
import base64
import pandas as pd
import urllib3



In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) 
#TO SUPPRESS THE SSL WARNING

In [ ]:
#GLOBAL VARS

#This sets hold all the relevant URLs of particular domains
global_set_of_urls = set()
global_set_of_pdf_urls = set()

# Names of the Output Folders
output_folder = "txt_files"
pdf_folder_for_sebi = "sebi_pdfs"
pdf_folder_for_cci = "cci_pdfs"
pdf_folder_for_rbi = "rbi_pdfs"

#dataframe to save the extracted pdf
df = pd.DataFrame(columns=["file_text"])

# List of URLs

# 1. Home Page for Competition Commission of India
cci_homepage_url = "https://www.cci.gov.in/"

# 2. Home Page for Securities and Exchange Board of India (SEBI)
# Note: The homepage URL for SEBI has been changed. I have included only the Menu's as all the relevant info/links are there.
sebi_homepage_url = "https://www.sebi.gov.in/js/menu.js"

# 3. Home Page for RBI
rbi_homepage_url = "https://www.rbi.org.in/"

# 4. Home Page of indiacode
indiacode_initializer_url = "https://www.indiacode.nic.in/handle/123456789/1362/browse?type=actyear" #instead of using the homepage, I am using the page where all the acts are listed by year


#Excluded words from the urls
excluded_suffixes_for_urls = ["pdf", "xlsx", "img", "docx", "png"]
excluded_words_from_urls_of_cci = ["events", "gallery", "ccijoclp"]
excluded_words_from_urls_of_indiacode = ["events", "gallery"]
excluded_words_from_urls_of_sebi = ["events", "gallery"]
excluded_words_from_urls_of_rbi = ["events", "gallery"]


excluded_urls_of_indiacode = [
    "https://www.indiacode.nic.in/handle/123456789/1362/browse?type=shorttitle",
    "https://www.indiacode.nic.in/handle/123456789/1362/browse?type=actno",
    "https://www.indiacode.nic.in/handle/123456789/1362/browse?type=dateissued",
    "https://www.indiacode.nic.in/handle/123456789/1362/browse?type=ministry",
    "https://www.indiacode.nic.in/handle/123456789/1362/browse?type=department"
]


In [4]:
def contains_excluded_word(input_string, excluded_words):
    for word in excluded_words:
        if word.lower() in input_string.lower():
            return True
    return False

In [5]:
#to get all the URLs of the mentioned URL
# Function -> to get all urls of CCI into a global set
def get_sub_urls_of_cci(url):
    # Note : CCI's website constantly giving SSL Error, hence I had to turn off their SSL Verification. Not a good solutions, need to work on that.
    read = requests.get(url,verify=False)
    html_content = read.text
    soup = BeautifulSoup(html_content,'html.parser')
    all_anchors_tags = soup.find_all('a')
    for link in all_anchors_tags:
        href_link = link.get('href')
        #the following if creates a set of urls which starts with http and are the links within the domain of "cci"(and not links of twitter, insta etc.)
        # Need to save all these strings into lists -> for better readability
        if href_link!=None and href_link.lower().startswith("http"):  
            if "cci" in href_link and not any(href_link.lower().endswith(suffix) for suffix in excluded_suffixes_for_urls) and not (contains_excluded_word(href_link,excluded_words_from_urls_of_cci)):
                    global_set_of_urls.add(href_link)


In [6]:
# Function -> to get all urls of SEBI into a global set
# SEBI Website has external js scripts which holds URL(inside anchor tags), hence need to take that into consideration
def get_sub_urls_of_sebi(url):
    read = requests.get(url,verify=False)
    html_content = read.text
    # Note : Need to add Exception handling to make it more robust.
    soup = BeautifulSoup(html_content,'html.parser')
    all_anchors_tags = soup.find_all('a')
    all_javascript_tags = soup.find_all('script')
    # To get all the relevant links from the <a> tags
    for link in all_anchors_tags:
        href_link = link.get('href')
        #the following if creates a set of urls which starts with http and are the links within the domain of "cci"(and not links of twitter, insta etc.)
        if(href_link!=None and href_link.startswith("https://")):
            if not (href_link.startswith("https://www.sebi")):
                continue
        else:
            href_link = urljoin(sebi_homepage_url,href_link)
        if href_link!=None and href_link.lower().startswith("http"):  
            if "sebi" in href_link and not any(href_link.lower().endswith(suffix) for suffix in excluded_suffixes_for_urls) and not (contains_excluded_word(href_link,excluded_words_from_urls_of_sebi)):
                global_set_of_urls.add(href_link)
    # For sebi we need to get a tags as well as <script> tags as, javascript files
    for link in all_javascript_tags:
        src_link = link.get('src')
        if(src_link!=None and src_link.startswith("https://")):
            if not (src_link.startswith("https://www.sebi")):
                continue
        else:
            src_link = urljoin(sebi_homepage_url,src_link)
        if src_link!=None and src_link.lower().startswith("http"):  
            if "sebi" in src_link and not any(src_link.lower().endswith(suffix) for suffix in excluded_suffixes_for_urls) and not (contains_excluded_word(src_link,excluded_words_from_urls_of_sebi)):
                global_set_of_urls.add(src_link)
            




In [7]:
def get_sub_urls_of_rbi(url):
    read = requests.get(url,verify=False)
    html_content = read.text
    # Note : Need to add Exception handling to make it more robust.
    soup = BeautifulSoup(html_content,'html.parser')
    all_anchors_tags = soup.find_all('a')
    
    # To get all the relevant links from the <a> tags
    for link in all_anchors_tags:
        href_link = link.get('href')
        #the following if creates a set of urls which starts with http and are the links within the domain of "cci"(and not links of twitter, insta etc.)
        if(href_link!=None and href_link.startswith("https://")):
            if not (href_link.startswith("https://www.rbi")):
                continue
        else:
            href_link = urljoin(rbi_homepage_url,href_link)#change the homepage url based on the domain
        if href_link!=None and href_link.lower().startswith("http"):  
            if "rbi" in href_link and not any(href_link.lower().endswith(suffix) for suffix in excluded_suffixes_for_urls) and not (contains_excluded_word(href_link,excluded_words_from_urls_of_rbi)):
                global_set_of_urls.add(href_link)

In [8]:
def get_sub_urls_of_indiacode(url):
    read = requests.get(url,verify=False)
    html_content = read.text
    # Note : Need to add Exception handling to make it more robust.
    soup = BeautifulSoup(html_content,'html.parser')
    all_anchors_tags = soup.find_all('a')

    indiacode_homepage_url = "https://www.indiacode.nic.in/"
    #href_link = urljoin(indiacode_homepage_url,href_link)
    
    # To get all the relevant links from the <a> tags
    for link in all_anchors_tags:
        href_link = link.get('href')
        #the following if creates a set of urls which starts with http and are the links within the domain of "cci"(and not links of twitter, insta etc.)
        if(href_link!=None and href_link.startswith("https://")):
            if not (href_link.startswith("https://www.india")):
                continue
        else:
            href_link = urljoin(indiacode_homepage_url,href_link)#change the homepage url based on the domain
        if(any(href_link.lower().endswith(excluded_url) for excluded_url in excluded_urls_of_indiacode)):
            continue
        if href_link!=None and href_link.lower().startswith("http"):  
            if not any(href_link.lower().endswith(suffix) for suffix in excluded_suffixes_for_urls) and not (contains_excluded_word(href_link,excluded_words_from_urls_of_indiacode)):
                global_set_of_urls.add(href_link)



In [ ]:
#NOT relevant web urls

#https://www.cci.gov.in/events/All/details/549 => these urls contain pdf with images -> not at all useful
#https://ccijournal.in/index.php/ccijoclp/article/view/16 => such links contains the articles which are case studies, book reviews etc. Hence not useful for our purpose
#gallery contains images of events => not useful
#the urls which ends with pdf, img, xlsx, docx -> NOT NECESSARY AT ALL when we are making global_set_of_sub_urls




In [9]:
# Objective : To create global list of all the URLs in the website
# The function recursively iterate over global_set and captures all the weblinks/url from all over the domain
def util_to_get_links_of_suburls(choice): 
    for link in global_set_of_urls.copy():
        if link not in visited_url:
            try:
                if choice == 1:
                    get_sub_urls_of_cci(link)#change this based on CCI or SEBI domain
                elif choice == 2:
                    get_sub_urls_of_sebi(link)
                elif choice == 3:
                    get_sub_urls_of_rbi(link)
                elif choice == 4:
                    get_sub_urls_of_indiacode(link)
                visited_url.add(link)
            except Exception as e:
                print("exception {e} occured for : {link}")

#print(len(global_set_of_urls))

In [11]:
# A Method to save all the collected list of Global URLs in CSV File

def make_csv_file_from_set(input_set,output_file):
    try:
        with open(output_file, 'w', newline='') as file:
            writer = csv.writer(file)
            
            # Write the elements from the set(global_set_of_urls) to the CSV file
            for item in input_set:
                writer.writerow([item])
    except Exception as e:
        print("Exception occured : ",e)
    else:
        print("CSV file created: ", output_file)


In [12]:
# A method that return soup of the url passed as arguments
# Update : To handle HttpConnectionTimeout, max tries reached Error => Exception handline as well as HTTPAdapter is added


import requests
from requests.adapters import HTTPAdapter

def soup_returner(url):
    soup = BeautifulSoup()
    try:
        session = requests.Session()
        retry = HTTPAdapter(max_retries=5)
        session.mount("http://", retry)
        session.mount("https://", retry)
        read = session.get(url,verify=False)
        html_content = read.text
        soup = BeautifulSoup(html_content,'html.parser')
    except Exception as e:
        print("URL ", url)
        print("Exception occured : ",e )
    return soup




In [13]:
# A method to fetch all the urls that ends with "pdf" 
# The method creates a global_set of all the "pdf" urls by using the global_set of urls

def make_set_of_pdf_links():
	#traverse one by one from global url set and for each url get the All links of pdf files under that webpage
	for url in global_set_of_urls:
		soup = soup_returner(url)
		only_anchor_tags = soup.find_all('a')
		for link in only_anchor_tags:
			href_link = link.get('href')
			#print(href_link)
			if href_link!=None and href_link.lower().endswith(".pdf"):
				global_set_of_pdf_urls.add(href_link)


In [ ]:
############# A METHOD TO CONVERT PDF CONTENT TO TEXT ##################################
def get_content_from_pdf(pdf_path):
    response = requests.get(pdf_path,verify=False)

    with io.BytesIO(response.content) as f:
        pdfreader = PdfFileReader(f,strict=False)
        information = pdfreader.getDocumentInfo()
        number_of_pages = pdfreader.getNumPages()

        print(information.title)

        #following code snippets to convert pdf_url into base64 encoding
        def convert_url_to_base64(url):
            sample_string = url
            sample_string_bytes = sample_string.encode("ascii") 
            base64_bytes = base64.b64encode(sample_string_bytes) 
            base64_string = base64_bytes.decode("ascii")
            return base64_string

        filename = convert_url_to_base64(pdf_path)
        relative_path= os.path.join(output_folder,filename)
        all_extracted_content = ""

        # for page in range(0,number_of_pages):
        #     page_text = pdfreader.getPage(page)
        #     all_extracted_content += page_text.extractText()

        # with open (relative_path+".txt", 'w', encoding="utf-8") as file:
        #     file.write(all_extracted_content)
    




In [15]:
# A Method to download all the pdf using the global_set of pdfs
# Each domain will have seperate folder directory
# Note : Need to automate changing the folder names according to the domain

# Note : Handle if the folder is not already created, it should also create it. 

def download_pdfs(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url, verify=False)
        response.raise_for_status()  # Raise an error for bad responses

        # Extract the file name from the URL
        filename = ""
        pdf_file_name = ""
        with io.BytesIO(response.content) as f:
            pdfreader = PdfFileReader(f,strict=False)
            information = pdfreader.getDocumentInfo()
            
            file_text = ""

            for page_num in range(pdfreader.getNumPages()):
                file_text += pdfreader.getPage(page_num).extractText()
            new_index = len(df)
            df.loc[new_index, 'file_text'] = file_text

        print(f"Written data frame for : ")

    except Exception as e:
        print(f"Error writing frame {url}: {e}")

In [ ]:
# The Method to extract text from scanned documents
# The method returns title and the text content of the pdf
import fitz  # PyMuPDF
from PIL import Image
import pytesseract

def extract_text_and_title(pdf_path):
    try:
        # Open the PDF file
        pdf_document = fitz.open(pdf_path)

        # Iterate through each page
        text_of_entire_file = ""
        for page_number in range(pdf_document.page_count):
            page = pdf_document.load_page(page_number)

            # Convert the page to an image
            image = page.get_pixmap()
            img = Image.frombytes("RGB", [image.width, image.height], image.samples)

            # Perform OCR on the image
            text = pytesseract.image_to_string(img)
            text_of_entire_file += text
            # Get the title (you may need to adjust this based on your PDF structure)
            title = page.get_text("title")

    except Exception as e:
        print(f"Error extracting text and title: {e}")

    return title,text_of_entire_file

In [22]:
choice = 1
#Choices for different website
#1. CCI Website
#2. SEBI
#3. RBI
#4. IndiaCode

if choice == 1:
    #Step by step scraping pipeling for CCI
    #1. Add initializer page to global_set
    print("###################### INITIALIZING ###################################")
    print("1. Creating a global set of URLs using HomePage URL of <### CCI ###>")
    global_set_of_urls.add(cci_homepage_url)
    #2. Get all the suburls using the initializer url(try to go through it 2 times)
    print("2. Searching Recursively for URLs")
    visited_url = set() #to keep track of urls already visited
    for i in range(0,2):
        util_to_get_links_of_suburls(choice)
    #3. (Optional Preprocessing) Use algo to remove unimportant urls based on some conditions
    print("3. Removing unimportant urls based on some conditions")
    #4. Save this list of global_set of URLS in csv file -> just for a backup
    print("4. Save this list of global_set of URLS in csv file -> just for a backup")
    globalurls_filename = "CCIglobalURL.csv"#output file to save the set of all global URLs
    make_csv_file_from_set(global_set_of_urls,globalurls_filename)
    #5. Get all the pdf urls using the global_set of urls
    print("5. Getting all the <pdf urls> using the global_set of urls")
    global_set_of_pdf_urls = set()
    make_set_of_pdf_links()
    #6. (Optional Preprocessing) Use algo to remove unimportant urls based on some conditions
    print("6. Removing unimportant pdf urls based on some conditions")
    #7. Save the list of global_set_of_pdf_urls into a csv file -> just for a backup
    print("7. Saving the list of global_set_of_pdf_urls into a csv file -> just for a backup")
    globalpdfurls_filename = "CCIglobalPDF_URL.csv"#output file to save the set of all global URLs
    make_csv_file_from_set(global_set_of_pdf_urls,globalpdfurls_filename)
    #8. Read each url of pdf link and save its content into a dataframe
    print("8. Saving the content of each pdf into a dataframe")
    for url in global_set_of_pdf_urls:
        download_pdfs(url)
    #9. Save this dataframe in csv file
    print("9. Copying the content into a csv file")
    df.to_csv('Extracted text of CCI PDFs.csv')
elif choice == 2:
    #1. Add initializer page to global_set
    global_set_of_urls.add(sebi_homepage_url)
    #2. Get all the suburls using the initializer url(try to go through it 2 times)
    visited_url = set() #to keep track of urls already visited
    for i in range(0,2):
        util_to_get_links_of_suburls(choice)
    #3. (Optional Preprocessing) Use algo to remove unimportant urls based on some conditions
        
    #4. Save this list of global_set of URLS in csv file -> just for a backup
    globalurls_filename = "SEBIglobalURL.csv"#output file to save the set of all global URLs
    make_csv_file_from_set(global_set_of_urls,globalurls_filename)
    #5. Get all the pdf urls using the global_set of urls
    global_set_of_pdf_urls = set()
    make_set_of_pdf_links()
    #6. (Optional Preprocessing) Use algo to remove unimportant urls based on some conditions
    #7. Save the list of global_set_of_pdf_urls into a csv file -> just for a backup
    globalpdfurls_filename = "SEBIglobalPDF_URL.csv"#output file to save the set of all global URLs
    make_csv_file_from_set(global_set_of_pdf_urls,globalpdfurls_filename)
    #8. Read each url of pdf link and save its content into a dataframe
    for url in global_set_of_pdf_urls:
        download_pdfs(url)
    #9. Save this dataframe in csv file
    df.to_csv('Extracted text of SEBI PDFs.csv')
elif choice == 3:
    #1. Add initializer page to global_set
    global_set_of_urls.add(rbi_homepage_url)
    #2. Get all the suburls using the initializer url(try to go through it 2 times)
    visited_url = set() #to keep track of urls already visited
    for i in range(0,2):
        util_to_get_links_of_suburls(choice)
    #3. (Optional Preprocessing) Use algo to remove unimportant urls based on some conditions
        
    #4. Save this list of global_set of URLS in csv file -> just for a backup
    globalurls_filename = "RBIglobalURL.csv"#output file to save the set of all global URLs
    make_csv_file_from_set(global_set_of_urls,globalurls_filename)
    #5. Get all the pdf urls using the global_set of urls
    global_set_of_pdf_urls = set()
    make_set_of_pdf_links()
    #6. (Optional Preprocessing) Use algo to remove unimportant urls based on some conditions
    #7. Save the list of global_set_of_pdf_urls into a csv file -> just for a backup
    globalpdfurls_filename = "RBIglobalPDF_URL.csv"#output file to save the set of all global URLs
    make_csv_file_from_set(global_set_of_pdf_urls,globalpdfurls_filename)
    #8. Read each url of pdf link and save its content into a dataframe
    for url in global_set_of_pdf_urls:
        download_pdfs(url)
    #9. Save this dataframe in csv file
    df.to_csv('Extracted text of RBI PDFs.csv')
elif choice == 4:
    #1. Add initializer page to global_set
    global_set_of_urls.add(indiacode_initializer_url)
    #2. Get all the suburls using the initializer url(try to go through it 2 times)
    visited_url = set() #to keep track of urls already visited
    for i in range(0,2):
        util_to_get_links_of_suburls(choice)
    #3. (Optional Preprocessing) Use algo to remove unimportant urls based on some conditions
        
    #4. Save this list of global_set of URLS in csv file -> just for a backup
    globalurls_filename = "IndiaCodeglobalURL.csv"#output file to save the set of all global URLs
    make_csv_file_from_set(global_set_of_urls,globalurls_filename)
    #5. Get all the pdf urls using the global_set of urls
    global_set_of_pdf_urls = set()
    make_set_of_pdf_links()

    #6. (Optional Preprocessing) Use algo to remove unimportant urls based on some conditions
    updated_global_set = set()
    for link in global_set_of_pdf_urls:
        if "repealed" not in link:
            indiacode_homepage_url = "https://www.indiacode.nic.in/"
            new_link = urljoin(indiacode_homepage_url,link)
            updated_global_set.add(new_link)

    global_set_of_pdf_urls = updated_global_set

    #7. Save the list of global_set_of_pdf_urls into a csv file -> just for a backup
    globalpdfurls_filename = "IndiaCodeglobalPDF_URL.csv"#output file to save the set of all global URLs
    make_csv_file_from_set(global_set_of_pdf_urls,globalpdfurls_filename)
    #8. Read each url of pdf link and save its content into a dataframe
    for url in global_set_of_pdf_urls:
        download_pdfs(url)
    #9. Save this dataframe in csv file
    df.to_csv('Extracted text of IndiaCode PDFs.csv')

KeyboardInterrupt: 